# Business performance analysis 
We will calculate such indicators as: Conversion, bounce, retention, LVT, ROI, CAC and build charts to make it easier to analyze.

# Step 1 - Loading data and preparing it for analysis

In [5]:
# importing modules

import pandas as pd
import seaborn
import numpy as np
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

In [6]:
# import visit data
visits = pd.read_csv('data/costs_info_short.csv')
visits.head(2)

,dt,Channel,costs
0,2019-05-01,FaceBoom,113.3
1,2019-05-02,FaceBoom,78.1


In [7]:
# rename table columns
visits = visits.rename(columns={'User Id': 'user_id',
                        'Region': 'region',
                        'Device': 'device',
                        'Channel': 'channel',
                        'Session Start': 'session_start',
                        'Session End': 'session_end'
                       })
visits.head(1)

,dt,channel,costs
0,2019-05-01,FaceBoom,113.3


In [8]:
# import order data
orders = pd.read_csv('data/orders_info_short.csv')
orders.head(2)

,User Id,Event Dt,Revenue
0,188246423999,2019-05-01 23:09:52,4.99
1,174361394180,2019-05-01 12:24:04,4.99


In [9]:
# rename table columns
orders = orders.rename(columns={'User Id': 'user_id',
                        'Event Dt': 'event_dt',
                        'Revenue': 'revenue'
                       })
orders.head(1)

,user_id,event_dt,revenue
0,188246423999,2019-05-01 23:09:52,4.99


In [10]:
# import cost data
costs = pd.read_csv('data/costs_info_short.csv')
costs.head(2)

,dt,Channel,costs
0,2019-05-01,FaceBoom,113.3
1,2019-05-02,FaceBoom,78.1


In [11]:
# rename table columns
costs = costs.rename(columns={'Channel': 'channel'
                       })
costs.head(1)

,dt,channel,costs
0,2019-05-01,FaceBoom,113.3


In [12]:
# проверим данные на наличие дубликатов

print('Number of duplicates в visits:', visits.duplicated().sum())
print('Number of duplicates в orders:', orders.duplicated().sum())
print('Number of duplicates в costs:', costs.duplicated().sum())

Количество дубликатов в visits: 0
Количество дубликатов в orders: 0
Количество дубликатов в costs: 0


In [13]:
# chcecking for gaps
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   dt       1800 non-null   object 
 1   channel  1800 non-null   object 
 2   costs    1800 non-null   float64
dtypes: float64(1), object(2)
memory usage: 42.3+ KB


In [14]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40212 entries, 0 to 40211
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   user_id   40212 non-null  int64  
 1   event_dt  40212 non-null  object 
 2   revenue   40212 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 942.6+ KB


In [15]:
costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   dt       1800 non-null   object 
 1   channel  1800 non-null   object 
 2   costs    1800 non-null   float64
dtypes: float64(1), object(2)
memory usage: 42.3+ KB


- no duplicates were found in the datasets,
- the number of records in all datasets is the same
- no passes found

# Step 2 - we will set functions for calculating and analyzing LTV, ROI, retention and conversion.